In [1]:
import os
import nibabel as nib
import numpy as np

def file_to_ndarray(filepath):
    # Check the file extension
    _, file_extension = os.path.splitext(filepath)
    
    try:
        if file_extension in ['.nii', '.nii.gz']:  # Handle gzipped or regular NIfTI files
            # Load the NIfTI file
            nii_img = nib.load(filepath)
            # Convert to ndarray
            data = nii_img.get_fdata()
            #print(f"Loaded NIfTI file: {filepath}")
        else:
            print("Unsupported file format.")
            return None
        
        return data
    
    except Exception as e:
        print(f"An error occurred while processing the file: {e}")
        return None

In [2]:
def compute_histogram(ndarray):
    
    flat_array = ndarray.flatten()

    # Define the bin edges from -4000 to 4000 with a bin size of 10
    bins = np.arange(-4000, 4001, 10)  # 2001 to include the endpoint 2000 in the last bin

    # Compute histogram
    histogram_values, bin_edges = np.histogram(flat_array, bins=bins)

    # Convert histogram values to list
    histogram_list = histogram_values.tolist()

    return histogram_list, bin_edges


In [3]:
import csv

def append_to_csv(file_path, file_name, list_to_append):
    # Open the file in append mode

    list_to_append = [file_name] + list_to_append

    with open(file_path, mode='a', newline='') as file:
        writer = csv.writer(file)
        # Write the list as the last row in the CSV
        writer.writerow(list_to_append)
        #print("Successfully to the CSV.")


In [4]:
import pandas as pd

def read_csv_as_dataframe(filepath):
    # Read the CSV file without headers
    df = pd.read_csv(filepath, header=None)

    # Use the first column as the header
    headers = df.iloc[:, 0]  # Extract the first column as headers
    df = df.iloc[:, 1:]      # Remove the first column from the df
    df = df.T
    df.columns = headers     # Set the extracted column as headers

    return df

In [5]:
import pandas as pd
import matplotlib.pyplot as plt

def plot_columns_of_dataframe(df, x_coords):
    # Check if the length of x_coords matches the number of rows in the DataFrame
    if len(x_coords) != len(df):
        raise ValueError("Length of x_coords must match the number of rows in the DataFrame")

    plt.figure(figsize=(10, 6))

    # Loop through each column in the DataFrame and plot
    for column in df.columns:
        plt.plot(x_coords, df[column], label=f'Column: {column}')

    plt.title('Voxels intensity distribution')
    plt.xlabel('Voxels intensity')
    plt.ylabel('Voxels count')
    #plt.legend()
    #plt.grid(True)
    plt.show()

In [17]:
import os

# Define the directory path where the .nii files are located
volume_folder = r"C:\Users\acer\Desktop\Project_TMJOA\Data\Open access data\Baseline_fixed"  # Replace with your actual directory path
csv_path = rf"{volume_folder}\volxel_intensity.csv"

nii_count = len([filename for filename in os.listdir(volume_folder) if filename.endswith('.nii.gz')])
print(f"There are {nii_count} .nii files in the {volume_folder}")

progress_count = 0

files = sorted(os.listdir(volume_folder))

# Loop through each file in the directory
error_log = []
for filename in files:

    if filename.endswith('.nii.gz'):
        
        # Progress bar
        progress_count += 1
        print(f"[Processing {progress_count} out of {nii_count}]")

        try:
            # Full path to the file
            file_path = os.path.join(volume_folder, filename)
            #print(f"Processing file: {filename}")

            # Read .nii file
            nii_img = nib.load(file_path)
            data = nii_img.get_fdata()

            # # Compute histogram
            histogram_values, bin_edges = compute_histogram(data)

            # # Write to csv
            append_to_csv(csv_path, filename, histogram_values)
        except Exception as e:
            print(f"Error processing {filename}: {e}")
            error_log.append(filename)

for file in error_log:
    print(file)


There are 144 .nii files in the C:\Users\acer\Desktop\Project_TMJOA\Data\Open access data\Baseline_fixed
[Processing 1 out of 144]
[Processing 2 out of 144]
[Processing 3 out of 144]
[Processing 4 out of 144]
[Processing 5 out of 144]
[Processing 6 out of 144]
[Processing 7 out of 144]
[Processing 8 out of 144]
[Processing 9 out of 144]
[Processing 10 out of 144]
[Processing 11 out of 144]
[Processing 12 out of 144]
[Processing 13 out of 144]
[Processing 14 out of 144]
[Processing 15 out of 144]
[Processing 16 out of 144]
[Processing 17 out of 144]
[Processing 18 out of 144]
[Processing 19 out of 144]
[Processing 20 out of 144]
[Processing 21 out of 144]
[Processing 22 out of 144]
[Processing 23 out of 144]
[Processing 24 out of 144]
[Processing 25 out of 144]
[Processing 26 out of 144]
[Processing 27 out of 144]
[Processing 28 out of 144]
[Processing 29 out of 144]
[Processing 30 out of 144]
[Processing 31 out of 144]
[Processing 32 out of 144]
[Processing 33 out of 144]
[Processing 3